# Notebook 1 : Préparation et nettoyage des données

## Chargement des données

In [1]:
import pandas as pd

file_path = "Collecte des données.xlsx"
xls = pd.ExcelFile(file_path)
sheet_names = xls.sheet_names
sheet_names

['PESTEL',
 '1 Production_Volaille',
 '2 Importations_Volaille',
 '3 Consommation_Volaille',
 '4 PIB_Economie',
 '5 Soutien_Bio',
 '6 Normes_Sanitaires',
 '7 Dependance_Import',
 '8 Population_Demographie']

## Nettoyage des données

## Nettoyage des pays avec moins de 2M d'habitants

In [2]:
# Identifier les pays à exclure via la feuille Population
df_pop = xls.parse("8 Population_Demographie")

# Filtrer uniquement pour l’année 2018
df_pop_2018 = df_pop[df_pop["Année"] == 2018]

# Nettoyage des noms (espaces avant/après)
df_pop_2018.loc[:, "Zone"] = df_pop_2018["Zone"].str.strip()

# Identifier les pays à exclure (population < 2M)
pays_exclus_pop = df_pop_2018[df_pop_2018["Valeur"] < 2_000_000]["Zone"].unique()

# Dans chaque feuille, .str.strip() sur la colonne Zone avant de filtrer
dfs = {}
for sheet in sheet_names:
    df = xls.parse(sheet)
    if "Zone" in df.columns:
        df["Zone"] = df["Zone"].str.strip()
        df = df[~df["Zone"].isin(pays_exclus_pop)]
    dfs[sheet] = df  # stocker le dataframe nettoyé

print("Pays exclus (population < 2M en 2018) :")
print(pays_exclus_pop[:50])


Pays exclus (population < 2M en 2018) :
['Andorre' 'Anguilla' 'Antigua-et-Barbuda' 'Antilles néerlandaises (ex)'
 'Aruba' 'Bahamas' 'Bahreïn' 'Barbade' 'Belize' 'Bermudes' 'Bhoutan'
 'Bonaire, Saint-Eustache et Saba' 'Brunéi Darussalam' 'Cabo Verde'
 'Chine - RAS de Macao' 'Chypre' 'Comores' 'Curaçao' 'Djibouti'
 'Dominique' 'Estonie' 'Eswatini' 'Fidji' 'Gibraltar' 'Grenade'
 'Groenland' 'Guadeloupe' 'Guam' 'Guinée équatoriale' 'Guinée-Bissau'
 'Guyana' 'Guyane française' 'Île de Man' 'Îles Anglo-Normandes'
 'Îles Caïmanes' 'Îles Cook' 'Îles Falkland (Malvinas)' 'Îles Féroé'
 'Îles Mariannes du Nord' 'Îles Marshall' 'Îles Salomon'
 'Îles Turques-et-Caïques' 'Îles Vierges américaines'
 'Îles Vierges britanniques' 'Îles Wallis-et-Futuna' 'Islande' 'Kiribati'
 'Lettonie' 'Liechtenstein' 'Luxembourg']


## Pivot des éléments et fusion des feuilles

In [3]:
def process_sheet(sheet_name, xls):
    df = xls.parse(sheet_name)
    if "Zone" in df.columns and "Élément" in df.columns:
        df = df[["Zone", "Élément", "Valeur"]]
        df = df.groupby(["Zone", "Élément"], as_index=False).mean()
        df_pivot = df.pivot(index="Zone", columns="Élément", values="Valeur")
        df_pivot.columns = [f"{sheet_name} - {col}" for col in df_pivot.columns]
        return df_pivot
    return pd.DataFrame()

processed_sheets = [process_sheet(name, xls) for name in sheet_names if name != "PESTEL"]
df = pd.concat(processed_sheets, axis=1)

# Nettoyage de l’index pays
df.index = df.index.str.strip()

# On nettoie aussi la liste des pays exclus
pays_exclus_pop = pd.Series(pays_exclus_pop).str.strip()

# Suppression des pays exclus selon leur population
df = df[~df.index.isin(pays_exclus_pop)]

print(f"Pays exclus : {len(pays_exclus_pop)}")
print(f"Pays restants : {df.shape[0]}")


df.head(5)

Pays exclus : 87
Pays restants : 161


,1 Production_Volaille - Animaux Producteurs/Abattus,1 Production_Volaille - Production,2 Importations_Volaille - Importations - quantité,3 Consommation_Volaille - Disponibilité alimentaire en quantité (kg/personne/an),4 PIB_Economie - Croissance annuelle US$,5 Soutien_Bio - Émissions (CH4),5 Soutien_Bio - Émissions (CO2eq) (AR5),5 Soutien_Bio - Émissions (CO2eq) venant de CH4 (AR5),5 Soutien_Bio - Émissions (CO2eq) venant de N2O (AR5),5 Soutien_Bio - Émissions (N2O),6 Normes_Sanitaires - Valeur,7 Dependance_Import - Disponibilité intérieure,7 Dependance_Import - Importations - quantité,8 Population_Demographie - Population totale
Zone,,,,,,,,,,,,,,
Afghanistan,30057.052632,2.421365e+04,13853.210526,1.965556,11.301551,361.448274,13044.603505,10120.551663,2924.051842,11.034158,51.658333,63.444444,37.444444,2.884577e+07
Afrique du Sud,822713.052632,1.307453e+06,121922.108158,36.037778,6.368262,623.807574,26413.151537,17466.612063,8946.539474,33.760526,17.400000,1971.888889,392.000000,5.090147e+07
Albanie,9278.789474,1.210428e+04,11492.599211,12.832222,9.036131,89.906889,3247.696642,2517.392905,730.303737,2.755863,38.391667,37.000000,22.222222,2.994856e+06
Algérie,276235.842105,2.762359e+05,1877.245556,7.371111,7.887150,227.987795,9823.003989,6383.658253,3439.345737,12.978663,20.441667,285.666667,1.888889,3.585847e+07
Allemagne,572440.105263,7.724907e+05,222035.812105,17.875556,3.540220,1387.588989,48899.375337,38852.491705,10046.883632,37.912768,3.741667,1575.333333,763.111111,8.152701e+07


## Nettoyage des données

In [4]:
# Suppression des pays à éviter selon l'ONU, l'UE et l'OFAC aux USA
df.index = df.index.str.strip()

pays_exclus = [
    "Corée du Nord", "Iran", "Syrie", "Russie", "Biélorussie", 
    "Venezuela", "Cuba", "Soudan du Sud", "Birmanie (Myanmar)", 
    "Afghanistan", "Yémen"
]

df = df[~df.index.isin(pays_exclus)]


In [5]:
threshold = int(0.7 * len(df))
df = df.dropna(axis=1, thresh=threshold)
df = df.dropna()
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 137 entries, Afrique du Sud to Éthiopie
Data columns (total 13 columns):
 #   Column                                                                            Non-Null Count  Dtype  
---  ------                                                                            --------------  -----  
 0   1 Production_Volaille - Animaux Producteurs/Abattus                               137 non-null    float64
 1   1 Production_Volaille - Production                                                137 non-null    float64
 2   2 Importations_Volaille - Importations - quantité                                 137 non-null    float64
 3   3 Consommation_Volaille - Disponibilité alimentaire en quantité (kg/personne/an)  137 non-null    float64
 4   4 PIB_Economie - Croissance annuelle US$                                          137 non-null    float64
 5   5 Soutien_Bio - Émissions (CH4)                                                   137 non-null    fl

## Ajout du Ratio entre la production de volaille et la population totale 

In [6]:
# Création d'une nouvelle variable : Production par habitant
col_production = "1 Production_Volaille - Production"
col_population = "8 Population_Demographie - Population totale"

# Création du ratio (production par habitant)
df["Production_Volaille_par_habitant"] = df[col_production] / df[col_population]

# Vérification rapide
df["Production_Volaille_par_habitant"].describe()

count    137.000000
mean       0.012667
std        0.012856
min        0.000063
25%        0.001833
50%        0.008668
75%        0.020074
max        0.059611
Name: Production_Volaille_par_habitant, dtype: float64

## Normalisation des données

In [7]:
# Mise à l'echelle des données avec la formule suivante :
# (Valeur - Moyenne)/Ecart-type

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
df_scaled = pd.DataFrame(scaler.fit_transform(df), index=df.index, columns=df.columns)

# Réintégration de la colonne Zone
df_scaled.insert(0, "Zone", df_scaled.index)
df_scaled.reset_index(drop=True, inplace=True)

df_scaled.head()

,Zone,1 Production_Volaille - Animaux Producteurs/Abattus,1 Production_Volaille - Production,2 Importations_Volaille - Importations - quantité,3 Consommation_Volaille - Disponibilité alimentaire en quantité (kg/personne/an),4 PIB_Economie - Croissance annuelle US$,5 Soutien_Bio - Émissions (CH4),5 Soutien_Bio - Émissions (CO2eq) (AR5),5 Soutien_Bio - Émissions (CO2eq) venant de CH4 (AR5),5 Soutien_Bio - Émissions (CO2eq) venant de N2O (AR5),5 Soutien_Bio - Émissions (N2O),7 Dependance_Import - Disponibilité intérieure,7 Dependance_Import - Importations - quantité,8 Population_Demographie - Population totale,Production_Volaille_par_habitant
0,Afrique du Sud,0.394038,0.371031,1.288359,1.346491,-0.557343,-0.059324,-0.012711,-0.059324,0.149565,0.149565,0.535300,1.534471,0.014736,1.016419
1,Albanie,-0.333372,-0.303248,-0.329257,-0.305517,0.291545,-0.341415,-0.356503,-0.341415,-0.387680,-0.387680,-0.328324,-0.387469,-0.288572,-0.673372
2,Algérie,-0.094647,-0.165758,-0.470106,-0.694295,-0.074048,-0.268459,-0.258921,-0.268459,-0.210540,-0.210540,-0.217333,-0.493153,-0.080505,-0.387490
3,Allemagne,0.170232,0.092562,2.754865,0.053518,-1.457196,0.344225,0.321000,0.344225,0.221515,0.221515,0.358300,3.463342,0.208633,-0.249166
4,Angola,-0.321316,-0.298886,0.882265,-0.361837,2.646009,-0.296769,-0.292960,-0.296769,-0.262413,-0.262413,-0.199529,1.020491,-0.162436,-0.919128


## Export du dataset préparé

In [8]:
df_scaled.to_csv("donnees_preparees.csv", index=False)